# **FIUBA Organización de Datos 75.06**
# **Introduccion a [Apache Spark](https://spark.apache.org/) usando [pySpark]((https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)**

## Arquitectura

En Spark, para poder realizar una tarea la comunicacion ocurre entre un **driver** y una serie de **executors** (ejecutores). El driver tiene **jobs** y para ejecutarlos se rompen en **tareas (tasks)** que se envian a los executors. Los resultados de esas tareas se envian denuevo al **driver**.  

En este IPython Notebook la funcion del driver de Spark se ejecuta dentro del kernel asociado al notebook. 

Al utilizar localmente o conectandose a un cluster, el software driver 
es **[PySpark shell](https://spark.apache.org/docs/latest/programming-guide.html#using-the-shell)**. Este contiene el main loop del programa que **creara datasets distribuidos (RDDs)** en el cluster y **aplicará operaciones (transformations & actions)** a esos datasets.

### Ejemplo de Cluster

En este diagrama podemos ver un ejemplo de un cluster, suponiendo que estamos corriendo una aplicacion o un determinada tarea en el cluster, se pueden ver en el grafico, marcados en violeta una serie de cores dedicados a ese job o aplicacion.

![executors](http://spark-mooc.github.io/web-assets/images/executors.png)

A alto nivel una aplicacion de Spark Consiste en un programa **driver** que **lanza** varias **operaciones paralelas en multiples executors JVMs que corren en un cluster o localmente en la misma maquina** (localmente puede correr cada uno en cada core disponible).

## SparkContext

Al utilizar Spark, uno crea una nueva aplicacion de Spark creando un `SparkContext`, el cual nos permitirá interactuar con el API de Spark. 

Cuando se crea el `SparkContext` se le pide al master algunos cores para poder ejecutar tasks. El master separar estos cores para esa aplicacion, no siendo utilizados por otras aplicaciones.

Al usar el entorno de lab desde el IPython Notebook el `SparkContext` esta creado en la variable `sc`

In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

In [ ]:
help(sc)

Para mas información sobre como inicializar el entorno de spark en un contexto de cluster ver [https://spark.apache.org/docs/latest/programming-guide.html#initializing-spark](https://spark.apache.org/docs/latest/programming-guide.html#initializing-spark)

## Construyendo RDDs

Los **RRDs (Resilent Distributed Datasets)**, son colecciones tolerantes a fallos cuyos elementos pueden ser operados en paralelo. Se pueden crear de a partir de colecciones o **a partir de external datasets (distribuidos o no)** que veremos con algunos ejemplos.

### Paralelizando una coleccion de python

Inicialmente creamos una coleccion de datos en python que usaremos para crear el RDD usando `sc.parallelize`

In [ ]:
## creamos 1000 enteros en una lista
integersList = range(1,1001)
#integersList
len(integersList)

In [ ]:
## Paralelizamos la coleccion utilizando 8 particiones o slices
## Esta operacion es una transformacion de datos en un RDD
## Dado que Spark usa lazy evaluation, no corren jobs de Spark
## hasta el momento
integersListRDD = sc.parallelize(integersList, 6)
type(integersListRDD)

In [ ]:
## podemos ver tambien otra informacion interesante del RDD
## el numero de particiones
integersListRDD.getNumPartitions()

In [ ]:
## el conjunto de transformaciones que se aplica
print(integersListRDD.toDebugString())

In [ ]:
## para ver mas metodos disponibles del RDD
help(integersListRDD)

## ¿Qué sucede en el cluster?: Información distribuida

En Spark los datasets son representados como una lista de entradas, la cual se rompe en distintas particiones, cada una guarda en un maquina distinta del cluster. 

Cada particion tiene un unico subconjunto de las entradas de la lista. La abstraccion que usa Spark para manipularlos se conoce como "Resilent Distributed Datasets" (RDDs) y una de las particularidades que tienen es que estan disponibles en memoria (de ser posible).

El escenario de nuestro ejemplo se veria de la siguiente forma si consideramos algunos executors

![partitions](http://spark-mooc.github.io/web-assets/images/partitions.png)

### Creando RDDs desde external Datasets

PySpark puede crear datasets distribuidos desde cualquier storage soportado por Hadoop (local filesystem, HDFS, Cassandra, HBase, AWS S3, etc). Soportando archivos de texto y otros tipos de formatos de entrada soportados por Hadoop ([SequenceFiles](http://hadoop.apache.org/docs/current/api/org/apache/hadoop/mapred/SequenceFileInputFormat.html) y [InputFormat](http://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/InputFormat.html)).

### Cargando un archivo de texto

Aprovechando que tenemos: [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). Vamos a convertir el archivo de texto en un RDD usando el método `SparkContext.textFile()`. 

In [ ]:
## creamos el RDD a partir de un archivo de texto
shakespeareRDD = sc.textFile('data/shakespeare.txt',8)
## aplicamos una accion para tomar los 15 items del RDD
shakespeareRDD.take(200)

## Transformaciones: `map()`

Hasta el momento **creamos datasets distribuidos que se han dividido en varias particiones, donde cada particion se encuentra en una maquina de nuestro cluster**, veamos que sucede al hacer una operacion a nuestro dataset.

Vamos a partir de la operacion mas usual que podemos llegar a querer hacer sobre nuestro **'por cada uno de los elementos del dataset hagamos una accion y apliquemos devolvamos un resultado'**. Esto se logra con una **transformacion `map()`** que aplicara la función f a cada resultado.

In [ ]:
integersListRDD.take(10)

In [ ]:
## aplicamos una transformacion map para restar a todos 1
## aplicamos la accion take para mostrar 10 resultados
# ver generacion de tuplas y uso de funciones de python en pyspark.
integersListRDD.map(lambda x: x*2).take(5)

In [ ]:
## aplicamos una transformacion map para restar a todos 1
## aplicamos la accion take para mostrar 10 resultados

integersListRDD.map(lambda x: (x**2, x, x-1)).take(5)

In [ ]:
## generamos una tupla que tenga el valor original, el anterior y siguiente
## aplicamos una transformacion map para restar a todos 1
## aplicamos la accion take para mostrar 10 resultados

integersListRDD.map(lambda a: (a, a-1, a+1)).take(10)

## ¿Qué sucede en el cluster?

Cuando ejecutamos `map()` en el dataset, un unico **stage** de tareas se lanza. Un **stage** es un grupo de tareas que van a realizar el mismo computo pero con distintos datos de entrada. Una tarea entonces se lanzara para cada una de las particiones.

En el grafico de abajo vemos que sucede entonces con cada una de las particiones al ejecutarse la tarea y el resultado que producen.

![foo](http://spark-mooc.github.io/web-assets/images/map.png)

## Acciones: `collect()`, `count()`

Luego de haber aplicado una transformacion en el RDD, podemos 
queres volver a obtener informacion en el **driver**, podemos utilizar una accion `collect()`, usualmente se utiliza despues de alguna operacion que limite la cantidad de resultados (`filter()`) para asegurarnos que el resultado devuelto entre en la memoria disponible del driver.

Ya usamos anteriormente otra accion `take()` para traer un numero de resultados al driver.

Volviendo a nuestro ejemplo con los numero enteros podemos:

In [ ]:
## aplicamos una transformacion map para restar a todos 1
## aplicamos la accion take para mostrar 10 resultados

subRDD = integersListRDD.map(lambda a: a-1)
print(subRDD.collect())


En el cluster sucede la siguiente situacion, al realizarse el `collect()`
devolviendo los resultados a SparkContext.

![collect](http://spark-mooc.github.io/web-assets/images/collect.png)

Otra accion interesante es `count()` que permite contar la cantidad de elementos en el RDD

In [ ]:
# por ejemplo para los RDD con los que estuvimos trabajando

print(integersListRDD.count())
print(subRDD.count())
print(shakespeareRDD.count())

## Mas Transformaciones: `filter()`

La transformacion `filter()` transformacion permite aplicar una funcion al evaluarse solamente emitira a la salida aquellos que la funcion filtro haya devuelto `True`.

In [ ]:
## obtener los numeros pares
print(integersListRDD.filter(lambda a: a % 2 == 0).take(10))
print(integersListRDD.filter(lambda a: a % 2 == 0).count())
print(integersListRDD.filter(lambda a: a == 2).take(10))

In [ ]:
# eliminar lineas vacias de the complete works of shakespeare
#shakespeareRDD.take(15)
shakespeareRDD.filter(lambda a: a != "").take(15)

In [ ]:
print(shakespeareRDD.filter(lambda a: a != "").count())
print(shakespeareRDD.count())

## Reduce

La accion `reduce()` reduce los elementos de un RDD a un unico valor al aplicar una funcion que toma dos parametros y retorna un unico valor.

La funcion planteada tiene que ser **conmutativa y asociativa**, ya que `reduce()` **es aplicado a nivel de particion y luego nuevamente para agregar resultados de particiones**. Si esto no se respeta, los resultados de `reduce()` seran inconsistentes.

In [ ]:
## la suma es asociativa y conmutativa
print(shakespeareRDD.reduce(lambda a, b: a + b))
print(shakespeareRDD.repartition(4).reduce(lambda a, b: a + b))

In [ ]:
## la resta no no asociativa y conmutativa
print(integersListRDD.reduce(lambda a, b: a - b))
print(integersListRDD.repartition(4).reduce(lambda a, b: a - b))

## Otras Acciones: `first()`,`takeOrdered()`,`top()`

Estas son otras acciones que pueden ser utiles para procesar informacion


In [ ]:
## first() devuelve el primer elemento del RDD
print(shakespeareRDD.first())

In [ ]:
## traemos los 3 elementos mas pequeños
print(integersListRDD.takeOrdered(3))
## traemos los 3 elementos mas grandes
print(integersListRDD.top(3))

In [ ]:
## se puede utilizar una funcion para fijar el orden en takeOrdered
## por ejemplo para revertirlo
integersListRDD.takeOrdered(3, lambda a: -a)

Otras acciones avanzadas para investigar son [`takeSample()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeSample) y [`countByValue()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByValue)

## Transformaciones Adicionales

### `flatMap()`

Similar a map, pero permite que cada item de entrada se mapee a cero o mas elementos de salida.

In [ ]:
simpleRDD = sc.parallelize([2, 3, 4])
print(simpleRDD.map(lambda x: [1, x]).collect())
print(simpleRDD.flatMap(lambda x: [1, x]).collect())

In [ ]:
listadelistas = [[1,2],[2,3],[5]]
rdd = sc.parallelize(listadelistas)
print(rdd.flatMap(lambda x: x).collect())

### Transformaciones para pair RDDs

Las siguientes transformaciones aplican a RDDs donde los elementos 
son tuplas del tipo `(key, value)`


### `reduceByKey()`

la transformacion `reduceByKey()` junta pares que tienen la misma key y aplica la funcion a los dos valores asociados a la vez. Esta opera primero aplicando la funcion a cada particion para las mismas claves y luego a traves de las particiones (similar al reduce).

### `groupByKey()`

Agrupa todos los elementos de una misma clave generando una clave con una lista con los elementos.

Ambas se pueden usar para resolver el mismo problema, pero reduceBykey es mas eficiente en datasets distribuidos grandes, dado que en este escenario Spark sabe que puede combinar output de un mismo key (en la misma maquina) antes de redistribuir datos entre los distintos nodos (shuffling).

In [ ]:
from operator import add

pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 1)])

# usamos mapValues para mejorar el formato de impresion
#print(pairRDD.reduceByKey(lambda a,b: a+b).collect())

# Diferentes formas de sumar por clave
# problemas de notacion python3
#print(pairRDD.groupByKey().map(lambda (k, v): (k, sum(v)))).collect())

# Using mapValues, which is recommended when they key doesn't change
#print(pairRDD.groupByKey().mapValues(lambda x: sum(x)).collect())

# reduceByKey is more efficient / scalable
#print(pairRDD.reduceByKey(add).collect())

#print(shakespeareRDD.flatMap(lambda a: a.split()))\
#.map(lambda a: (len(a), 1))\
#.take(100)

Otras transformaciones interesantes para investigar son [combineByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.combineByKey) y [foldByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.foldByKey).

## Ejercicio: Buscando la linea de máxima longitud

In [ ]:
# generando tuplas para obtener la linea 
# de maxima longitud de todo el texto

result = shakespeareRDD.reduce(lambda a,b: a if len(a) > len(b) else b)
print(result)